# Arquivo para gerar a filtragem dos dados de mortes do SivepGripe

Importa as bibliotecas

In [1]:
import pandas as pd
import numpy as np

Le o dataset que contem os dados, e realiza a filtragem para que contenha apenas os que são consideradas mortes por covid
(segundo o dicionario dos dados, quando as colunas Evolucao for igual a 2, e a classificacao final igual a 5)

In [2]:
df = pd.read_csv(r'SRAGHospitalizado_2020_12_07.csv',sep=';')

df_morte = df[df.EVOLUCAO == 2]
df_morte = df_morte[df.CLASSI_FIN == 5]

C:\Users\luank\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (15,18,60,62,63,64,92,94,106,108,117,121,123) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\luank\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


Realiza a divisão pelas idades e a seleção das colunas que serão utilizadas

In [3]:
categoria = pd.cut(df.NU_IDADE_N, bins=[0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,99])

df_morte.insert(19, 'Idade', categoria)

df_morte['Idade'].value_counts()
df_morte = df_morte[['EVOLUCAO', 'ID_MUNICIP', 'CO_MUN_NOT', 'Idade']]

df_aux = df_morte.copy()
df_morte

,EVOLUCAO,ID_MUNICIP,CO_MUN_NOT,Idade
30,2.0,SAO BERNARDO DO CAMPO,354870,"(75, 80]"
31,2.0,RIO DE JANEIRO,330455,"(30, 35]"
43,2.0,SAO PAULO,355030,"(70, 75]"
50,2.0,RIBEIRAO PIRES,354330,"(55, 60]"
51,2.0,ITAPECERICA DA SERRA,352220,"(70, 75]"
...,...,...,...,...
999936,2.0,TERESINA,221100,"(55, 60]"
999938,2.0,CAMACAN,290560,"(80, 99]"
999967,2.0,BRASILIA,530010,"(75, 80]"
999971,2.0,SAO PAULO,355030,"(65, 70]"


Realiza o agrupamento dos dados por municipio e por idade. Realiza o ajuste deles, e salva em um novo csv.

In [4]:
df_morte = df_morte.groupby([df_morte['ID_MUNICIP'], df_morte['Idade']]).count()
df_morte = df_morte.reset_index()

t = 'ZE DOCA'

df_aux.loc[df_aux['ID_MUNICIP'] == t, 'CO_MUN_NOT'].iloc[0]


for index, row in df_morte.iterrows():
    cid = row['ID_MUNICIP']
    cod = int(df_aux.loc[df_aux['ID_MUNICIP'] == cid, 'CO_MUN_NOT'].iloc[0])
    df_morte.at[index,'CO_MUN_NOT'] = int(cod)

df_morte = df_morte.replace(np.nan, 0)
df_morte = df_morte.rename(columns = {df_morte.columns[2]: 'Casos'})

df_morte.to_csv(r'mortes_idade.csv')
df_morte

,ID_MUNICIP,Idade,Casos,CO_MUN_NOT
0,ABADIA DOS DOURADOS,"(0, 5]",NaN,310010.0
1,ABADIA DOS DOURADOS,"(5, 10]",NaN,310010.0
2,ABADIA DOS DOURADOS,"(10, 15]",NaN,310010.0
3,ABADIA DOS DOURADOS,"(15, 20]",NaN,310010.0
4,ABADIA DOS DOURADOS,"(20, 25]",NaN,310010.0
...,...,...,...,...
39418,ZE DOCA,"(60, 65]",2.0,211400.0
39419,ZE DOCA,"(65, 70]",3.0,211400.0
39420,ZE DOCA,"(70, 75]",5.0,211400.0
39421,ZE DOCA,"(75, 80]",3.0,211400.0


In [5]:
df_morte[df_morte['ID_MUNICIP'] == 'Rio de Janeiro']

,ID_MUNICIP,Idade,Casos,CO_MUN_NOT
